In [145]:
!pip install torch torchvision
!pip install torch-geometric

In [146]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch.nn import Linear, Dropout
from torch_geometric.datasets import Amazon

In [147]:
# Importar el dataset de PyTorch Geometric
dataset = Amazon(root=".", name="Computers")

data = dataset[0]

In [148]:
class GCN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = F.dropout(x, p=0.1, training=self.training)
        h = self.gcn1(h, edge_index)
        h = F.relu(h)
        h = F.dropout(h, p=0.1, training=self.training)
        h = self.gcn2(h, edge_index)
        return F.log_softmax(h, dim=1)

def fit(self, data, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.parameters(), lr=0.01)

    self.train()
    for epoch in range(epochs + 1):
        optimizer.zero_grad()
        out = self(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 20 == 0:
            val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
            val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')


In [149]:
GCN.fit = fit

@torch.no_grad()
def test(self, data):
    self.eval()
    out = self(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

GCN.test = test


In [150]:
# Fijar la semilla
torch.manual_seed(42)

In [151]:
# Total d nodos
num_nodes = data.num_nodes

In [152]:
perm = torch.randperm(num_nodes)

In [153]:
# Proporciones de división
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

In [154]:
# Indices de división
train_end = int(train_ratio * num_nodes)
val_end = train_end + int(val_ratio * num_nodes)

In [155]:
data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

data.train_mask[perm[:train_end]] = True
data.val_mask[perm[train_end:val_end]] = True
data.test_mask[perm[val_end:]] = True

def accuracy(y_pred, y_true):
    """Calcular la precisión."""
    return torch.sum(y_pred == y_true) / len(y_true)

gcn = GCN(dataset.num_features, 32, dataset.num_classes)
print(gcn)

GCN(
  (gcn1): GCNConv(767, 32)
  (gcn2): GCNConv(32, 10)
)


In [156]:
gcn.fit(data, epochs=180)

Epoch   0 | Train Loss: 2.262 | Train Acc: 17.23% | Val Loss: 2.28 | Val Acc: 14.69%
Epoch  20 | Train Loss: 1.357 | Train Acc: 64.71% | Val Loss: 1.36 | Val Acc: 62.84%
Epoch  40 | Train Loss: 0.857 | Train Acc: 75.73% | Val Loss: 0.89 | Val Acc: 74.84%
Epoch  60 | Train Loss: 0.613 | Train Acc: 82.09% | Val Loss: 0.65 | Val Acc: 82.69%
Epoch  80 | Train Loss: 0.501 | Train Acc: 86.04% | Val Loss: 0.56 | Val Acc: 85.45%
Epoch 100 | Train Loss: 0.452 | Train Acc: 86.95% | Val Loss: 0.50 | Val Acc: 86.40%
Epoch 120 | Train Loss: 0.407 | Train Acc: 88.32% | Val Loss: 0.47 | Val Acc: 87.42%
Epoch 140 | Train Loss: 0.388 | Train Acc: 88.47% | Val Loss: 0.46 | Val Acc: 87.71%
Epoch 160 | Train Loss: 0.366 | Train Acc: 89.36% | Val Loss: 0.43 | Val Acc: 88.80%
Epoch 180 | Train Loss: 0.359 | Train Acc: 89.65% | Val Loss: 0.43 | Val Acc: 88.22%


In [157]:
# Test
acc = gcn.test(data)
print(f'GCN test accuracy: {acc*100:.2f}%')

GCN test accuracy: 88.37%
